In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from random import choice as rc
import re
from typing import List
import pyarrow as pa
import pickle
import string

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10,8)

# Functions

In [ ]:
def join_str_with_underscore(str_list):
    a = str_list[0]
    for i in str_list[1:]:
        a = a + f"_{i}" 
    return a

def get_top_name_from_number(top_number:int, subdomain_name:str)->str:
    if re.match(r"Sphere", subdomain_name):
        return ['Bf0I1', 'Bf1S2', 'Bf1S2'][top_number]
    elif re.match(r"Cylinder", subdomain_name):
        return ['Bf0I1', 'Bf1S1', 'Bf2I1'][top_number]
    elif re.match(r"FilledCylinder", subdomain_name):
        return ['Bf0I1', 'Bf1B2Radial', 'Bf1B2'][top_number]
    else:
        raise Exception(f"{subdomain_name=} not recognized!")

def get_domain_name(col_name):
  def AMR_domains_to_decimal(subdoamin_name):
    # SphereC28.0.1
    a = subdoamin_name.split(".")
    # a = [SphereC28,0,1]
    decimal_rep = a[0]+"."
    # decimal_rep = SphereC28.
    for i in a[1:]:
      decimal_rep = decimal_rep + i
    # decimal_rep = SphereC28.01
    return decimal_rep

  if "on" in col_name:
    return AMR_domains_to_decimal(col_name.split(" ")[-1])
  if "." in col_name:
    return AMR_domains_to_decimal(col_name.split(" ")[-1])
  elif "_" in col_name:
    return col_name.split("_")[0]
  elif "MinimumGridSpacing" in col_name:
    return col_name.split("[")[-1][:-1]
  else:
    return col_name
    # raise Exception(f"{col_name} type not implemented in return_sorted_domain_names")

def filtered_domain_names(domain_names, filter):
    return [i for i in domain_names if re.match(filter, get_domain_name(i))]

def sort_spheres(sphere_list,reverse=False):
    if len(sphere_list) == 0:
        return []
    if "SphereA" in sphere_list[0]:
        return sorted(sphere_list, key=lambda x: float(get_domain_name(x).lstrip('SphereA')), reverse=reverse)
    elif "SphereB" in sphere_list[0]:
        return sorted(sphere_list, key=lambda x: float(get_domain_name(x).lstrip('SphereB')), reverse=reverse)
    elif "SphereC" in sphere_list[0]:
        return sorted(sphere_list, key=lambda x: float(get_domain_name(x).lstrip('SphereC')), reverse=reverse)
    elif "SphereD" in sphere_list[0]:
        return sorted(sphere_list, key=lambda x: float(get_domain_name(x).lstrip('SphereD')), reverse=reverse)
    elif "SphereE" in sphere_list[0]:
        return sorted(sphere_list, key=lambda x: float(get_domain_name(x).lstrip('SphereE')), reverse=reverse)

def return_sorted_domain_names(domain_names):

  FilledCylinderCA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}CA')
  CylinderCA = filtered_domain_names(domain_names, r'Cylinder.{0,2}CA')
  FilledCylinderEA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}EA')
  CylinderEA = filtered_domain_names(domain_names, r'Cylinder.{0,2}EA')
  SphereA = sort_spheres(filtered_domain_names(domain_names, 'SphereA'), reverse=True)
  CylinderSMA = filtered_domain_names(domain_names, r'CylinderS.{0,2}MA')
  FilledCylinderMA = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}MA')

  FilledCylinderMB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}MB')
  CylinderSMB = filtered_domain_names(domain_names, r'CylinderS.{0,2}MB')
  SphereB = sort_spheres(filtered_domain_names(domain_names, 'SphereB'), reverse=True)
  CylinderEB = filtered_domain_names(domain_names, r'Cylinder.{0,2}EB')
  FilledCylinderEB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}EB')
  CylinderCB = filtered_domain_names(domain_names, r'Cylinder.{0,2}CB')
  FilledCylinderCB = filtered_domain_names(domain_names, r'FilledCylinder.{0,2}CB')

  SphereC = sort_spheres(filtered_domain_names(domain_names, 'SphereC'), reverse=False)
  SphereD = sort_spheres(filtered_domain_names(domain_names, 'SphereD'), reverse=False)
  SphereE = sort_spheres(filtered_domain_names(domain_names, 'SphereE'), reverse=False)


  combined_columns = [FilledCylinderCA, CylinderCA, FilledCylinderEA, CylinderEA, SphereA, CylinderSMA, FilledCylinderMA, FilledCylinderMB, CylinderSMB, SphereB, CylinderEB, FilledCylinderEB, CylinderCB, FilledCylinderCB, SphereC, SphereD, SphereE]
  combined_columns = [item for sublist in combined_columns for item in sublist]

  # Just append the domains not following any patterns in the front. Mostly domains surrounding sphereA for high spin and mass ratios
  combined_columns_set = set(combined_columns)
  domain_names_set = set()
  for i in domain_names:
    domain_names_set.add(i)
  subdomains_not_sorted = list(domain_names_set - combined_columns_set)
  return subdomains_not_sorted+combined_columns

In [ ]:
def limit_by_col_val(min_val,max_val,col_name,df):
  filter = (df[col_name]>=min_val) &(df[col_name] <=max_val)
  return df[filter]

def read_dat_file(file_name):
  cols_names = []
  # Read column names
  with open(file_name,'r') as f:
      lines = f.readlines()
      for line in lines:
        if "#" not in line:
          # From now onwards it will be all data
          break
        elif "=" in line:
          if ("[" not in line) and ("]" not in line):
             continue
          cols_names.append(line.split('=')[-1][1:-1].strip())
        else:
          continue

  return pd.read_csv(file_name,sep="\s+",comment="#",names=cols_names)

def read_dat_file_single_bh(file_name):
  cols_names = []
  # Find the max number of columns
  with open(file_name,'r') as f:
    max_columns = max(len(line.split()) for line in f if not line.startswith('#'))
  return pd.read_csv(file_name,sep="\s+",comment="#",header=None,names=[str(i) for i in np.arange(-1,max_columns)]).rename(columns={'-1': 't'})


def find_subdomains(path:Path):
  subdomain_set = set()
  for i in path.iterdir():
    if i.is_dir():
      subdomain_set.add(i.stem)

  return list(subdomain_set)

def find_topologies(path:Path):
  topologies_set = set()
  for i in path.iterdir():
    if i.is_file():
      topologies_set.add(i.stem.split("_")[0])

  return list(topologies_set)

def find_dat_file_names(path:Path):
  file_name_set = set()
  for i in path.iterdir():
    if i.is_file():
      file_name_set.add(i.stem.split("_")[1])

  return list(file_name_set)

def get_top_name_and_mode(name):
  # Bf0I1(12 modes).dat -> Bf0I1, 12
  top_name = name.split("(")[0]
  mode = int(name.split("(")[-1].split(" ")[0])
  return top_name,mode

def find_highest_modes_for_topologies(path:Path):
  highest_mode_dict = {}
  for i in path.iterdir():
    if i.is_file():
      top_name, mode = get_top_name_and_mode(i.stem)
      if top_name in highest_mode_dict:
        if highest_mode_dict[top_name] < mode:
          highest_mode_dict[top_name] = mode
      else:
        highest_mode_dict[top_name] = mode

  return highest_mode_dict

def make_mode_dataframe(path:Path):
  highest_mode_dict = find_highest_modes_for_topologies(path)
  top_dataframe_list = {i:[] for i in highest_mode_dict}

  for i in path.iterdir():
    for top_name in highest_mode_dict:
      if (top_name+"(") in i.stem:
        top_dataframe_list[top_name].append(read_dat_file(i))

  top_mode_df_dict = {}
  for i,df_list in top_dataframe_list.items():
    result = pd.concat(df_list, ignore_index=True)

    # Remove duplicates based on 't' column (keep first occurrence)
    # result = result.drop_duplicates(subset='t', keep='first')

    # Sort by 't' and reset index
    top_mode_df_dict[i] = result.sort_values('t').reset_index(drop=True)
  return top_mode_df_dict

def filter_columns(cols: List[str], include_patterns: List[str] = None, 
                  exclude_patterns: List[str] = None) -> List[str]:
    """
    Filter a list of column names using include and exclude regex patterns.
    
    Args:
        cols: List of column names to filter
        include_patterns: List of regex patterns to include (if None, includes all)
        exclude_patterns: List of regex patterns to exclude (if None, excludes none)
    
    Returns:
        List of filtered column names
    
    Examples:
        >>> cols = ['age_2020', 'age_2021', 'height_2020', 'weight_2021']
        >>> filter_columns(cols, ['age_.*'], ['.*2021'])
        ['age_2020']
    """
    # Handle None inputs
    include_patterns = include_patterns or ['.*']
    exclude_patterns = exclude_patterns or []
    
    # First, get columns that match any include pattern
    included_cols = set()
    for pattern in include_patterns:
        included_cols.update(
            col for col in cols 
            if re.search(pattern, col)
        )
    
    # Then remove any columns that match exclude patterns
    for pattern in exclude_patterns:
        included_cols = {
            col for col in included_cols 
            if not re.search(pattern, col)
        }
    
    return sorted(list(included_cols))

def chain_filter_columns(cols: List[str], include_patterns: List[str] = None, 
                        exclude_patterns: List[str] = None) -> List[str]:
    """
    Filter columns sequentially using chained include and exclude regex patterns.
    Each pattern filters from the result of the previous pattern.
    
    Args:
        cols: List of column names to filter
        include_patterns: List of regex patterns to include (if None, includes all)
        exclude_patterns: List of regex patterns to exclude (if None, excludes none)
    
    Returns:
        List of filtered column names
    
    Examples:
        >>> cols = ['age_2020_q1', 'age_2020_q2', 'age_2021_q1', 'height_2020_q1']
        >>> chain_filter_columns(cols, ['age_.*', '.*q1'], ['.*2021.*'])
        ['age_2020_q1']
    """
    # Handle None inputs
    include_patterns = include_patterns or ['.*']
    exclude_patterns = exclude_patterns or []
    
    # Start with all columns
    filtered_cols = set(cols)
    
    # Apply include patterns sequentially
    for pattern in include_patterns:
        filtered_cols = {
            col for col in filtered_cols 
            if re.search(pattern, col)
        }
    
    # Apply exclude patterns sequentially
    for pattern in exclude_patterns:
        filtered_cols = {
            col for col in filtered_cols 
            if not re.search(pattern, col)
        }
    
    return sorted(list(filtered_cols))

def sort_by_coefs_numbers(col_list:List[str]):
  with_coef_list = []
  without_coef_list = []
  for col in col_list:
    if 'coef' not in col:
      without_coef_list.append(col)
    else:
      with_coef_list.append(col)
  return without_coef_list+sorted(with_coef_list, key=lambda x: int(x.split("_")[-1][4:]))

def get_extreme_coef_for_each_domain(df, min_or_max='min'):
    col_names = df.columns
    subdomains = set([i.split("_")[0] for i in col_names]) - set(['t(M)'])
    exterme_coef = {'t(M)':df['t(M)']}
    for sd in subdomains:
        sd_cols = [i for i in col_names if f"{sd}_" in i]
        if min_or_max == 'max':
            exterme_coef[sd] = df[sd_cols].max(axis=1)
        elif min_or_max == 'min':
            exterme_coef[sd] = df[sd_cols].min(axis=1)
        else:
            raise Exception(f"Only supported values of min_or_max are min and max and not {min_or_max=}")

    return pd.DataFrame(exterme_coef)

def load_power_diagonistics(PowDiag_path:Path):
  pow_diag_dict = {}
  for sd in find_subdomains(PowDiag_path):
    pow_diag_dict[sd] = {}
    sd_path = PowDiag_path/f"{sd}.dir"

    psi_pd = make_mode_dataframe(sd_path/f"Powerpsi.dir")
    kappa_pd = make_mode_dataframe(sd_path/f"Powerkappa.dir")
    # For each subdomain save things by topology
    for top in find_topologies(sd_path):
      pow_diag_dict[sd][top]={}
      psi_pd_sorted_cols = sort_by_coefs_numbers(psi_pd[top].columns.to_list())
      pow_diag_dict[sd][top][f'psi_ps'] = psi_pd[top][psi_pd_sorted_cols]

      kappa_pd_sorted_cols = sort_by_coefs_numbers(kappa_pd[top].columns.to_list())
      pow_diag_dict[sd][top][f'kappa_ps'] = kappa_pd[top][kappa_pd_sorted_cols]

      for dat_file in find_dat_file_names(sd_path):
        pow_diag_dict[sd][top][f'{dat_file}'] = read_dat_file(sd_path/f"{top}_{dat_file}.dat")
  
  return pow_diag_dict


def load_power_diagonistics_flat(PowDiag_path:Path, reload:bool=False, return_df:bool=True, load_dat_files_only:list[str]=None):
  
  cache_data = PowDiag_path/"pandas.pkl"
  if cache_data.exists():
    if not reload:
        with open(cache_data, 'rb') as f:
            pow_diag_dict = pickle.load(f)
            print(f"Loaded from cache {cache_data}")
        return pow_diag_dict

  # Same as load_power_diagonistics but no nested dicts. This makes it easy to filter
  pow_diag_dict = {}
  for sd in find_subdomains(PowDiag_path):
    sd_path = PowDiag_path/f"{sd}.dir"

    for top in find_topologies(sd_path):
      for dat_file in find_dat_file_names(sd_path):
        if load_dat_files_only is not None:
            for allowed_dat_files in load_dat_files_only:
                if re.search(allowed_dat_files, dat_file) is None:
                    continue
                else:
                    print(dat_file)
        pow_diag_dict[f'{sd}_{top}_{dat_file}'] = read_dat_file(sd_path/f"{top}_{dat_file}.dat")
    if load_dat_files_only is not None:
        print(sd_path)
        continue

    psi_pd = make_mode_dataframe(sd_path/f"Powerpsi.dir")
    kappa_pd = make_mode_dataframe(sd_path/f"Powerkappa.dir")
    # For each subdomain save things by topology
    for top in find_topologies(sd_path):
      psi_pd_sorted_cols = sort_by_coefs_numbers(psi_pd[top].columns.to_list())
      pow_diag_dict[f'{sd}_{top}_psi_ps'] = psi_pd[top][psi_pd_sorted_cols]

      kappa_pd_sorted_cols = sort_by_coefs_numbers(kappa_pd[top].columns.to_list())
      pow_diag_dict[f'{sd}_{top}_kappa_ps'] = kappa_pd[top][kappa_pd_sorted_cols]

    print(sd_path)
  
  if return_df:
    # This can be definitely merged with the stuff above but it's fast enough anyways
    flat_dict = {}
    flat_dict['t'] = pow_diag_dict[rc(list(pow_diag_dict.keys()))]['t']
    for key,item in pow_diag_dict.items():
      for col in item.columns:
        if 't' == col:
          continue 
        else:
          flat_dict[f"{key}_{col}"] = item[col]

    flat_df = pd.DataFrame(flat_dict)
    with open(cache_data, 'wb') as f:
        pickle.dump(flat_df, f)
        print(f"Cached at {cache_data}")

    return flat_df


  return pow_diag_dict


def convert_series_to_coeff_df(data, top_num):
    irr_top_regex = 0
    match top_num:
        case 0:
            irr_top_regex = r'Bf0' # First top
        case 1:
            irr_top_regex = r'Bf1(S\d|B2R)' # Second top, S2 for spheres, B2 for filled cylinders
        case 2:
            irr_top_regex = r'((Bf1S2|Bf1B2_)|Bf2)' # Thrid top S2 for spheres, B2 radial for filled cylinders
        case _:
            raise Exception(f"{top_num=} should be one of [0,1,2]")
        

    indices = set(data.index) - set(['t(M)'])
    indices = set([i for i in indices if re.search(irr_top_regex,i)])

    subdomains = set([i.split("_")[0] for i in indices]) 
    # The coefs are sorted?? Do not assume but they are

    data_dict = {sd:{} for sd in subdomains}

    coef_number = lambda x: int(x.split("_")[-1][4:])

    max_coef = 0
    for sd in subdomains:
        for idx in indices:
            if f"{sd}_" in idx:
                if pd.notna(data[idx]):
                    data_dict[sd][coef_number(idx)] = data[idx]
        data_dict[sd] = dict(sorted(data_dict[sd].items()))
        if len(data_dict[sd]) > max_coef:
            max_coef = len(data_dict[sd])


    pd_data = pd.DataFrame(data_dict)
    return pd_data

def series_closest_to_time(t,df):
    time_index = np.where(df['t(M)']>t)[0][0]
    time = df['t(M)'][time_index]
    return time, df.iloc[time_index].copy()




In [ ]:
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/6_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Powerpsi.dir/Bf0I1(19 modes).dat")
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Powerpsi.dir/Bf0I1(20 modes).dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Bf0I1_HighestThirdConvergenceFactor.dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/6_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Bf0I1_TruncationError.dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics/SphereC10.dir/Powerpsi.dir/Bf1S2(15 modes).dat")
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_HP32_AF_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Powerkappa.dir/Bf0I1(15 modes).dat")

# BBH

In [ ]:
# data = read_dat_file(file)
h5_paths = []
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/h5_files_Lev5'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/h5_files_Lev3'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/h5_files_Lev4'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/h5_files_Lev5'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev6/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev0/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev1/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev2/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev3/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev4/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev5/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev0/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev3'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/16_set1_L3/h5_files_Lev3/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP28/h5_files_Lev3'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/17_set_main_q3_18_L3/h5_files_Lev3'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/17_set1_q3_18_L3/h5_files_Lev3'))
# h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/17_set3_q3_18_L3/h5_files_Lev3'))

# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/20_set1_L3_fine_cylinders/temp_h5_files_Lev3"))
# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/21_set1_L3_fine_cylinders_minExtent/temp_h5_files_Lev3"))
# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/22_set1_L1_long/h5_files_Lev1"))
# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/h5_files_Lev3"))

# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/26_main_L6_long/h5_files_8000M_Lev6"))
# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/26_set1_L6_long/h5_files_Lev6"))
# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/26_segs/set1_L6_AK_S2_L20_no_Rmin/h5_files_AK_Lev6"))

# h5_paths.append(Path("/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L3/h5_files_Lev3"))
# h5_paths.append(Path("/resnick/groups/sxs/hchaudha/spec_runs/RM_0_test/h5_files_Lev6"))
# h5_paths.append(Path("/resnick/groups/sxs/hchaudha/spec_runs/RM_1_Lev3/h5_files_Lev3"))
# h5_paths.append(Path("/resnick/groups/sxs/hchaudha/spec_runs/30_RM_set1_L1/h5_files_Lev1"))

# h5_paths.append(Path("/groups/sxs/hchaudha/spec_runs/del"))

h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/'))
h5_paths.append(Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev6/'))

domain = 'SphereA0'
domain = 'SphereA4'
# domain = 'FilledCylinderMA0'
# domain = 'FilledCylinderMA1'
# domain = 'CylinderSMB0.0'
domain = 'CylinderSMA1.0'
# domain = 'CylinderEA0.0.0'
# domain = 'CylinderEA1.0.0'
# domain = 'FilledCylinderEA0'
# domain = 'FilledCylinderEA1'
# domain = 'CylinderCA0.0.0'
# domain = 'CylinderCA1.0.0'
# domain = 'FilledCylinderCA0'
# domain = 'FilledCylinderCA1'
# domain = 'CylinderCA1.0.0'
# domain = 'SphereC0'
# domain = 'SphereC6'
# domain = 'SphereC0'
# domain = 'SphereC6'

if "RM" in str(h5_paths[-1]):
    psi_or_kappa = 'RMpsi'
else:
    psi_or_kappa = 'psi'
# psi_or_kappa = 'kappa'

folder_paths = [Path(f"{h5_path}/extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir") for h5_path in h5_paths]
top_data_dict = {join_str_with_underscore(str(folder_path).split("/")[-5:-3]+[domain]) : make_mode_dataframe(folder_path) for folder_path in folder_paths}
print(top_data_dict[list(top_data_dict.keys())[0]].keys())

In [ ]:
top_name = get_top_name_from_number(0,domain)
top_name = get_top_name_from_number(1,domain)
top_name = get_top_name_from_number(2,domain)


# top_name = list(top_data.keys())[0]
t_min = 0
t_min = 2000
t_min = 3000
# t_min = 9700
t_max = 40000
# t_max = 8400

min_coef = -1
# min_coef = 10
max_coef = 100
# max_coef = -4
plot_slice = slice(0,None)
# plot_slice = slice(0,-4)
# plot_slice = slice(-1,None)

title = ""
# plt.figure(figsize=(10, 7))
has_more_than_one = len(list(top_data_dict.keys())) > 1
style_list = ['-',':','--','-.']
num_legends = 0
single_legend,max_col = True,-1
for key_num,A,key in zip(range(100),string.ascii_uppercase,top_data_dict):

    # if 'Lev5' not in key:
    #     continue

    plt.gca().set_prop_cycle(None)
    data = top_data_dict[key][top_name]
    data = limit_by_col_val(t_min,t_max,'t',data)
    data = data.dropna(axis=1, how='all')  # Some columns will have just nans remove those
    column_names = data.columns[1:]
    visual_data = data[column_names]

    cols_to_use = [i for i in data.columns if 't' not in i]
    df = np.log10(data[cols_to_use])
    df['row_min'] = df.min(axis=1)
    df['row_max'] = df.max(axis=1)
    df['row_mean'] = df.mean(axis=1)
    df['row_std'] = df.std(axis=1)

    # plt.plot(df['row_min'])
    # plt.plot(df['row_mean'])
    # plt.plot(df['row_max'])
    # plt.plot(df['row_std'])
    for i in cols_to_use[plot_slice]:
        coef_num = int(i[4:])
        if coef_num < min_coef or coef_num > max_coef:
            continue
        # plt.plot(data['t'], df[f'{i}'])
        label = f"{i}"
        if has_more_than_one:
            if (max_col < coef_num):
                label = f"{A}: {i}"
                num_legends = num_legends + 1
            else:
                label = None
            max_col = max(coef_num,max_col)
        plt.plot(data['t'], df[f'{i}'], label = label, linestyle = style_list[key_num%len(style_list)])

    if has_more_than_one:
        title  = title+ f"{style_list[key_num%len(style_list)]}  {A}: "
    title = title+f'{key}_{top_name}\n'

if num_legends > 20:
    plt.legend(ncol=int(np.ceil(num_legends/20)))
else:
    plt.legend()
plt.tight_layout()
plt.title(title[:-1])
plt.xlabel('t(M)')
plt.ylabel(f'Power {psi_or_kappa}')
plt.tight_layout() 
plt.grid(False)
# plt.ylim(-16,-12)
plt.show()
# x = data['t']
# y = df['row_mean']
# y_err = df['row_std']
# plt.errorbar(x, y, yerr=y_err, fmt='-o', label='Data with Error Bars', ecolor='red', capsize=4)



In [ ]:
top_name = get_top_name_from_number(0,domain)
top_name = get_top_name_from_number(1,domain)
# top_name = get_top_name_from_number(2,domain)

# top_name = list(top_data.keys())[0]
t_min = 0
# t_min = 800
t_min = 1400
t_min = 3000
t_min = 9000
t_max = 40000
# t_max = 4000

min_coef = -1
# min_coef = 7
max_coef = 100
# max_coef = 10
plot_slice = slice(0,None)
if 'f0' in top_name:
    plot_slice = slice(-1,None) # use for cheby last mode
elif 'S2' in top_name:
    plot_slice = slice(-5,-4) # use for S2 5th from last
else:
    plot_slice = slice(0,None) # No idea what to do

title = ""
# plt.figure(figsize=(10, 7))
has_more_than_one = len(list(top_data_dict.keys())) > 1
style_list = ['-',':','--','-.']
num_legends = 0
single_legend,max_col = True,0
for key_num,A,key in zip(range(100),string.ascii_uppercase,top_data_dict):
    # plt.gca().set_prop_cycle(None)
    data = top_data_dict[key][top_name]
    data = limit_by_col_val(t_min,t_max,'t',data)
    data = data.dropna(axis=1, how='all')  # Some columns will have just nans remove those
    column_names = data.columns[1:]
    visual_data = data[column_names]

    cols_to_use = [i for i in data.columns if 't' not in i]
    df = np.log10(data[cols_to_use])
    df['row_min'] = df.min(axis=1)
    df['row_max'] = df.max(axis=1)
    df['row_mean'] = df.mean(axis=1)
    df['row_std'] = df.std(axis=1)

    # plt.plot(df['row_min'])
    # plt.plot(df['row_mean'])
    # plt.plot(df['row_max'])
    # plt.plot(df['row_std'])
    for i in cols_to_use[plot_slice]:
        coef_num = int(i[4:])
        if coef_num < min_coef or coef_num > max_coef:
            continue
        # plt.plot(data['t'], df[f'{i}'])
        label = f"{key}_{i}"
        plt.plot(data['t'], df[f'{i}'], label = label, linestyle = style_list[key_num%len(style_list)])

    # if has_more_than_one:
    #     title  = title+ f"{style_list[key_num%len(style_list)]}  {A}: "
    # title = title+f'{key}_{top_name}\n'
    title = f' {domain}:{top_name}'

if num_legends > 20:
    plt.legend(ncol=int(np.ceil(num_legends/20)))
else:
    plt.legend()
plt.tight_layout()
plt.title(title[:-1])
plt.xlabel('t(M)')
plt.ylabel(f'Power {psi_or_kappa}')
plt.tight_layout() 
plt.grid(False)
plt.show()
# x = data['t']
# y = df['row_mean']
# y_err = df['row_std']
# plt.errorbar(x, y, yerr=y_err, fmt='-o', label='Data with Error Bars', ecolor='red', capsize=4)

#### APS plots

In [ ]:
top_name = get_top_name_from_number(0,domain)
# top_name = get_top_name_from_number(1,domain)
# top_name = get_top_name_from_number(2,domain)


# top_name = list(top_data.keys())[0]
t_min = 0
t_min = 800
t_min = 3000
# t_min = 7000
t_max = 40000
# t_max = 4000

min_coef = -1
# min_coef = 7
max_coef = 100
# max_coef = 10
plot_slice = slice(0,None)
# plot_slice = slice(-1,None)

title = ""
# plt.figure(figsize=(10, 7))
has_more_than_one = len(list(top_data_dict.keys())) > 1
style_list = ['-',':','--','-.']
num_legends = 0
single_legend,max_col = True,0
for key_num,A,key in zip(range(100),string.ascii_uppercase,top_data_dict):
    with plt.style.context("ggplot"):
        plt.rcParams["figure.figsize"] = (8, 5.5)
        plt.gca().set_prop_cycle(None)
        data = top_data_dict[key][top_name]
        data = limit_by_col_val(t_min,t_max,'t',data)
        data = data.dropna(axis=1, how='all')  # Some columns will have just nans remove those
        column_names = data.columns[1:]
        visual_data = data[column_names]

        cols_to_use = [i for i in data.columns if 't' not in i]
        df = np.log10(data[cols_to_use])

        for i in cols_to_use[plot_slice]:
            coef_num = int(i[4:])
            if coef_num < min_coef or coef_num > max_coef:
                continue
            label = f"$a_{{{i[4:]}}}$"
            plt.plot(data['t'], np.power(10,df[f'{i}']), label = label, linestyle = style_list[key_num%len(style_list)])
        
        plt.yscale('log')

        title = title+f'{key}_{top_name}\n'

if num_legends > 20:
    plt.legend(ncol=int(np.ceil(num_legends/20)))
else:
    plt.legend()
plt.tight_layout()


if domain == "SphereC6":
    front = "Wave Zone, "
else:
    front = "Near the horizon, "

if psi_or_kappa == 'psi':
    title = front+r"norm($g_{ab}$)"
else:
    title = front+r"norm($\partial g_{ab}$)"

plt.title(title)
plt.xlabel('t(M)')
plt.ylabel(f'abs(coeffs)')
plt.tight_layout() 
plt.grid(False)

if domain == "SphereC6":
    plt.savefig(Path("/groups/sxs/hchaudha/scripts/report/figures")/f"spec_wave_{psi_or_kappa}.png", dpi=600)
else:
    plt.savefig(Path("/groups/sxs/hchaudha/scripts/report/figures")/f"spec_near_bh_{psi_or_kappa}.png", dpi=600)

plt.show()
# x = data['t']
# y = df['row_mean']
# y_err = df['row_std']
# plt.errorbar(x, y, yerr=y_err, fmt='-o', label='Data with Error Bars', ecolor='red', capsize=4)

# BH

In [ ]:
# data = read_dat_file(file)
dat_paths = []
# dat_paths.append(Path('/groups/sxs/hchaudha/spec_runs/single_bh/17_zero_spin_AMR/Lev5_265/'))
# dat_paths.append(Path('/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/Lev5_9565_longer'))
dat_paths.append(Path('/resnick/groups/sxs/hchaudha/spec_runs/single_bh_res/0_ghce_test/Lev7_4665'))

domain = 'SphereD0'
domain = 'SphereE49'

top_name = get_top_name_from_number(0,domain)
# top_name = get_top_name_from_number(1,domain)
# top_name = get_top_name_from_number(2,domain)

psi_or_kappa = 'kappa'
psi_or_kappa = 'psi'
psi_or_kappa = 'RMpsi'

folder_paths = [Path(f"{dat_path}/Run/AMRDiagnostics/{top_name}Power{psi_or_kappa}_{domain}.dat") for dat_path in dat_paths]
top_data_dict = {}
for folder_path in folder_paths:
    key_name = str(folder_paths).split("/")[-4]
    top_data_dict[key_name] = read_dat_file_single_bh(folder_path)
    top_data_dict[key_name] = top_data_dict[key_name].rename(columns={'time': 't'})
print(top_data_dict[list(top_data_dict.keys())[0]].keys())

In [ ]:
# top_name = list(top_data.keys())[0]
t_min = -1
# t_min = 800
# t_min = 3000
# t_min = 7000
t_max = 40000
# t_max = 4000

min_coef = -1
# min_coef = 7
max_coef = 100
# max_coef = 10
plot_slice = slice(0,None)
# plot_slice = slice(-1,None)

title = ""
# plt.figure(figsize=(10, 7))
has_more_than_one = len(list(top_data_dict.keys())) > 1
style_list = ['-',':','--','-.']
num_legends = 0
single_legend,max_col = True,0

plt.rcParams["figure.figsize"] = (6,6)
plt.rcParams["figure.autolayout"] = True
for key_num,A,key in zip(range(100),string.ascii_uppercase,top_data_dict):

    # if 'Lev5' not in key:
    #     continue

    plt.gca().set_prop_cycle(None)
    data = top_data_dict[key]
    data = limit_by_col_val(t_min,t_max,'t',data)
    data = data.dropna(axis=1, how='all')  # Some columns will have just nans remove those
    column_names = data.columns[1:]
    visual_data = data[column_names]

    cols_to_use = [i for i in data.columns if 't' not in i]
    df = np.log10(data[cols_to_use])
    df['row_min'] = df.min(axis=1)
    df['row_max'] = df.max(axis=1)
    df['row_mean'] = df.mean(axis=1)
    df['row_std'] = df.std(axis=1)

    # plt.plot(df['row_min'])
    # plt.plot(df['row_mean'])
    # plt.plot(df['row_max'])
    # plt.plot(df['row_std'])
    for i in cols_to_use[plot_slice]:
        coef_num = int(i)
        if coef_num < min_coef or coef_num > max_coef:
            continue
        # plt.plot(data['t'], df[f'{i}'])
        label = f"{i}"
        if has_more_than_one:
            if (max_col < coef_num):
                label = f"{A}: {i}"
                num_legends = num_legends + 1
            else:
                label = None
            max_col = max(coef_num,max_col)
        plt.plot(data['t'], df[f'{i}'], label = label, linestyle = style_list[key_num%len(style_list)])

    if has_more_than_one:
        title  = title+ f"{style_list[key_num%len(style_list)]}  {A}: "
    title = title+f'{key}_{domain}_{top_name}\n'

if num_legends > 20:
    plt.legend(ncol=int(np.ceil(num_legends/20)))
else:
    plt.legend()
plt.tight_layout()
plt.title(title[:-1])
plt.xlabel('t(M)')
plt.ylabel(f'Power {psi_or_kappa}')
plt.tight_layout() 
plt.grid(False)
plt.show()
# x = data['t']
# y = df['row_mean']
# y_err = df['row_std']
# plt.errorbar(x, y, yerr=y_err, fmt='-o', label='Data with Error Bars', ecolor='red', capsize=4)



In [ ]:
AMR_diag_paths = Path("/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/Lev5_9565_3/Run/AMRDiagnostics")
AMR_diag_paths = Path("/groups/sxs/hchaudha/spec_runs/single_bh/20_zero_spin_AMR_L5_10000M/Lev5_765/Run/AMRDiagnostics")

files = list(AMR_diag_paths.glob("*.dat"))
domains = set()
for file in files:
    domain = file.stem.split("_")[1]
    domains.add(domain)

pow_dict = {}
for domain in domains:
    for var in ['psi','kappa']:
        for top in ['Bf0I1','Bf1S2']:
            file_name = f"{top}Power{var}_{domain}.dat"
            file_path = AMR_diag_paths/f"{file_name}"

            pow_dict[f'{domain}_{top}_TruncationError_GridDiagPower{var}'] = read_dat_file_single_bh(file_path)


flat_dict = {}
flat_dict['t'] = pow_dict[rc(list(pow_dict.keys()))]['t']
for key,item in pow_dict.items():
    for col in item.columns:
        if 't' == col:
            continue 
        else:
            flat_dict[f"{key}_{col}"] = np.log10(item[col])

flat_df = pd.DataFrame(flat_dict)




# load all of power diagnostics

### df flat

In [ ]:
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set3_L6/h5_files_Lev6/extracted-PowerDiagnostics')
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev3/extracted-PowerDiagnostics')
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev3/extracted-PowerDiagnostics')
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev4/extracted-PowerDiagnostics')
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev5/extracted-PowerDiagnostics')
# PowDiag_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/extracted-PowerDiagnostics')
# PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/16_set1_L3/h5_files_Lev3/extracted-PowerDiagnostics")
# PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/22_set1_L3_long/h5_files_Lev3/extracted-PowerDiagnostics")
PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/26_set1_L6_long/h5_files_Lev6/extracted-PowerDiagnostics")
# PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/26_main_L6_long/h5_files_8000M_Lev6/extracted-PowerDiagnostics")
# PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/del/extracted-PowerDiagnostics")
flat_df = load_power_diagonistics_flat(PowDiag_path)

In [ ]:
include_list = [ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef']
include_list = [  r'TruncationError_', r'kappa']
# include_list = [  r'TruncationError_', r'psi']

topologies,topdim = [r'Bf0'],0 # First top
topologies,topdim = [r'Bf1(S\d|B2R)'],1 # Second top, S2 for spheres, B2 for filled cylinders
topologies,topdim = [r'((Bf1S2|Bf1B2_)|Bf2)'],2 # Thrid top S2 for spheres, B2 radial for filled cylinders

exclude_list = [ ]
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=flat_df.columns.tolist(),
    include_patterns=include_list+topologies,
    exclude_patterns=exclude_list
)

# You can then use these columns to filter your DataFrame
filtered_cols = sort_by_coefs_numbers(filtered_cols)
filtered_df = flat_df[filtered_cols].copy()
filtered_df.loc[:, 't(M)'] = flat_df['t']
filtered_df

In [ ]:
minT = 0
# minT = .2
# minT = 90
# minT = 1200
# minT = 3000
# minT = 3500
minT = 4000
minT = 8000

maxT = 40000
# maxT = 1530
# maxT = 3500
# maxT = 300
# maxT = 800
maxT = 8500

title = "".join(filtered_df.columns.to_list()[0].split("_")[1:])
data = limit_by_col_val(minT,maxT,'t(M)',filtered_df)
data = data.iloc[::1].dropna(axis=1, how='all')
data = data.rename(columns={i:""+i.split("_")[0] for i in data.columns})

domain_col_list = chain_filter_columns(
    cols=data.columns.tolist(),
    include_patterns=[ r'.*'],
    # include_patterns=[ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef'],
    # exclude_patterns=[],
    exclude_patterns=[r't\(M\)',r'SphereC']
)
visual_data = data[domain_col_list]
# visual_data = np.log10(visual_data)
# print(visual_data.columns)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names)
# print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -16.060739785509355 , -5.2211449021336955
# vmin_log,vmax_log = -10.267013515417048 , -7.6772687168580624
# vmin_log,vmax_log = -9 , None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

print(len(domain_col_list))
plt.figure(figsize=(15, 10))
# plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = 1
ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{title}')

# Determine colorbar ticks that align with the fixed vmin and vmax
# tick_vals = np.linspace(vmin_log, vmax_log, num=5)

# Set these ticks on the colorbar
# colorbar.set_ticks(tick_vals)

# Convert ticks back to the original scale for labeling
# colorbar.set_ticklabels([f'{10**val:.2e}' for val in tick_vals])

plt.ylabel('t(M)')
# plt.title(f'{PowDiag_path.parent},   top_dim={topdim}')
plt.tight_layout() 
plt.grid(False)
plt.show()

In [ ]:
include_list = [ r'SphereA0' ,r'Bf0I1', 'Number']
exclude_list = [  r'coef']
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=flat_df.columns.tolist(),
    include_patterns=include_list,
    exclude_patterns=exclude_list
)

# You can then use these columns to filter your DataFrame
filtered_cols = ['t']+sort_by_coefs_numbers(filtered_cols)
filtered_df = flat_df[filtered_cols]
filtered_df.columns

In [ ]:
plt.figure(figsize=(12, 8))
for col in filtered_df.columns:
  if 't' == col:
    continue
  plt.plot(filtered_df['t'], filtered_df[col],label=col)
  # plt.plot(filtered_df['t'], np.log10(np.abs(filtered_df[col])),label=col)
plt.legend()
plt.xlabel('t')
# plt.ylabel('Convergence Factor')
plt.show()

### power in last unfiltered mode

In [ ]:
include_list = [ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef']
include_list = [r'psi', r'HighestUnfiltered']
# include_list = [ r'kappa_ps', r'coef']
# include_list = [ r'psi_ps', r'coef']
# include_list = [  r'TruncationError_', r'kappa']
# include_list = [  r'TruncationError_', r'psi']

topologies,topdim = [r'Bf0'],0 # First top
topologies,topdim = [r'Bf1(S\d|B2R)'],1 # Second top, S2 for spheres, B2 for filled cylinders
topologies,topdim = [r'((Bf1S2|Bf1B2_)|Bf2)'],2 # Thrid top S2 for spheres, B2 radial for filled cylinders

exclude_list = [ ]
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=flat_df.columns.tolist(),
    include_patterns=include_list+topologies,
    exclude_patterns=exclude_list
)


# You can then use these columns to filter your DataFrame
filtered_cols = sort_by_coefs_numbers(filtered_cols)
filtered_df = flat_df[filtered_cols].copy()
filtered_df.loc[:, 't(M)'] = flat_df['t']
filtered_df

In [ ]:
minT = 0
minT = .2
# minT = 90
# minT = 1200
# minT = 3000
# minT = 3500
# minT = 7500

maxT = 40000
# maxT = 1530
# maxT = 3500
# maxT = 300
# maxT = 800
# maxT = 2

psi_or_kappa = None
if 'psi_' in filtered_cols[0]:
    psi_or_kappa = 'psi'
else:
    psi_or_kappa = 'kappa'

data = filtered_df.copy()
colorbar_label = f"{psi_or_kappa} :  Power in highest unfiltered mode"

data = limit_by_col_val(minT,maxT,'t(M)',data)

domain_col_list = chain_filter_columns(
    cols=data.columns.tolist(),
    include_patterns=[ r'.*',],
    # include_patterns=[ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef'],
    # exclude_patterns=[]
    exclude_patterns=[r't\(M\)']
)

visual_data = data[domain_col_list]
# visual_data = np.log10(visual_data)
# visual_data = visual_data>-14
# print(visual_data.columns)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names)
# print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -16.060739785509355 , -5.2211449021336955
# vmin_log,vmax_log = -10.267013515417048 , -7.6772687168580624
# vmin_log,vmax_log = -9 , None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

print(len(domain_col_list))
plt.figure(figsize=(15, 10))
# plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split("_")[0] for i in column_names], 
    rotation=90
)

ytick_step = 1
ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{colorbar_label}')

# Determine colorbar ticks that align with the fixed vmin and vmax
# tick_vals = np.linspace(vmin_log, vmax_log, num=5)

# Set these ticks on the colorbar
# colorbar.set_ticks(tick_vals)

# Convert ticks back to the original scale for labeling
# colorbar.set_ticklabels([f'{10**val:.2e}' for val in tick_vals])

plt.ylabel('t(M)')
include_list = [r'psi', r'HighestUnfiltered']
plt.title(f'{PowDiag_path.parent},   top_dim={topdim}')
plt.tight_layout() 
plt.grid(False)
plt.show()

### Min/Max power in coeffs/modes

Use the plotting above if you just want top get the highest non filtered mode. This one loads all the coeffs and allows for more processing (for now only min and max are implemented which is not that useful)

In [ ]:
include_list = [ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef']
# include_list = [ r'kappa_ps', r'coef']
include_list = [ r'psi_ps', r'coef']
# include_list = [  r'TruncationError_', r'kappa']
# include_list = [  r'TruncationError_', r'psi']

topologies,topdim = [r'Bf0'],0 # First top
topologies,topdim = [r'Bf1(S\d|B2R)'],1 # Second top, S2 for spheres, B2 for filled cylinders
# topologies,topdim = [r'((Bf1S2|Bf1B2_)|Bf2)'],2 # Thrid top S2 for spheres, B2 radial for filled cylinders

exclude_list = [ ]
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=flat_df.columns.tolist(),
    include_patterns=include_list+topologies,
    exclude_patterns=exclude_list
)


# You can then use these columns to filter your DataFrame
filtered_cols = sort_by_coefs_numbers(filtered_cols)
filtered_df = flat_df[filtered_cols].copy()
filtered_df.loc[:, 't(M)'] = flat_df['t']
filtered_df

In [ ]:
minT = 0
minT = .2
# minT = 90
minT = 1200
# minT = 3000
# minT = 3500
# minT = 7500

maxT = 40000
# maxT = 1530
# maxT = 3500
# maxT = 300
# maxT = 800
# maxT = 2

min_or_max = 'max'
min_or_max = 'min'
psi_or_kappa = None
if 'psi_' in filtered_cols[0]:
    psi_or_kappa = 'psi'
else:
    psi_or_kappa = 'kappa'

data = get_extreme_coef_for_each_domain(filtered_df, min_or_max=min_or_max)
colorbar_label = f"{psi_or_kappa} : {min_or_max} power coeff"

data = limit_by_col_val(minT,maxT,'t(M)',data)

domain_col_list = chain_filter_columns(
    cols=data.columns.tolist(),
    include_patterns=[ r'.*',],
    # include_patterns=[ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef'],
    # exclude_patterns=[]
    exclude_patterns=[r't\(M\)']
)

visual_data = data[domain_col_list]
visual_data = np.log10(visual_data)
# visual_data = visual_data>-14
# print(visual_data.columns)
# Plot using imshow

column_names = list(visual_data.columns)
# column_names = [i.split(" ")[-1] for i in visual_data.columns]
column_names = return_sorted_domain_names(column_names)
# print(column_names)

vmin_log,vmax_log = None,None
# vmin_log,vmax_log = -16.060739785509355 , -5.2211449021336955
# vmin_log,vmax_log = -10.267013515417048 , -7.6772687168580624
# vmin_log,vmax_log = -9 , None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

print(len(domain_col_list))
plt.figure(figsize=(15, 10))
# plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = 1
ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{colorbar_label}')

# Determine colorbar ticks that align with the fixed vmin and vmax
# tick_vals = np.linspace(vmin_log, vmax_log, num=5)

# Set these ticks on the colorbar
# colorbar.set_ticks(tick_vals)

# Convert ticks back to the original scale for labeling
# colorbar.set_ticklabels([f'{10**val:.2e}' for val in tick_vals])

plt.ylabel('t(M)')
plt.title(f'{PowDiag_path.parent},   top_dim={topdim}')
plt.tight_layout() 
plt.grid(False)
plt.show()

### The power spectrum at a given time

In [ ]:
# include_list = [ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef']
# include_list = [r'psi_ps', r'coef']
# include_list = [ r'kappa_ps', r'coef']
# include_list = [  r'TruncationError_', r'kappa']
include_list = [  r'TruncationError_', r'psi']

topologies,topdim = [],'all' # First top
# topologies,topdim = [r'Bf0'],0 # First top
# topologies,topdim = [r'Bf1(S\d|B2R)'],1 # Second top, S2 for spheres, B2 for filled cylinders
# topologies,topdim = [r'((Bf1S2|Bf1B2_)|Bf2)'],2 # Thrid top S2 for spheres, B2 radial for filled cylinders

exclude_list = [ ]
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=flat_df.columns.tolist(),
    include_patterns=include_list+topologies,
    exclude_patterns=exclude_list
)

# You can then use these columns to filter your DataFrame
filtered_cols = sort_by_coefs_numbers(filtered_cols)
filtered_df = flat_df[filtered_cols].copy()
filtered_df.loc[:, 't(M)'] = flat_df['t']
filtered_df

In [ ]:
time = 3500
time = 7500
time = 0
irr_top_num = 0
remove_first_n_coeffs = 0
# remove_first_n_coeffs = 6
time,series = series_closest_to_time(time,filtered_df)
data = convert_series_to_coeff_df(series,irr_top_num)

psi_or_kappa = None
if 'psi_' in filtered_cols[0]:
    psi_or_kappa = 'psi'
else:
    psi_or_kappa = 'kappa'

colorbar_label = f"{psi_or_kappa} : power coeff"


# domain_col_list = chain_filter_columns(
#     cols=data.columns.tolist(),
#     include_patterns=[ r'.*',],
#     # include_patterns=[ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef'],
#     # exclude_patterns=[]
#     exclude_patterns=[r't\(M\)']
# )

visual_data = np.log10(data)

column_names = list(visual_data.columns)
column_names = return_sorted_domain_names(column_names)
visual_data = visual_data[column_names].iloc[remove_first_n_coeffs:,:]

vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)

plt.figure(figsize=(15, 10))
# plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data, 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

plt.yticks(
    ticks=np.arange(0, len(visual_data), 1), 
    labels=np.arange(remove_first_n_coeffs, remove_first_n_coeffs+ len(visual_data), 1), 
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{colorbar_label}')
plt.ylabel('Power in Coeffs')
plt.title(f'{PowDiag_path.parent}\n Time={time:.2f} , {irr_top_num=}')
plt.tight_layout() 
plt.grid(False)
plt.show()

In [ ]:
time = 3500
time = 7500
irr_top_num = 2
remove_first_n_coeffs = 0
remove_first_n_coeffs = 6
time,series = series_closest_to_time(time,filtered_df)
data = convert_series_to_coeff_df(series,irr_top_num)

psi_or_kappa = None
if 'psi_' in filtered_cols[0]:
    psi_or_kappa = 'psi'
else:
    psi_or_kappa = 'kappa'

colorbar_label = f"{psi_or_kappa} : power coeff"


# domain_col_list = chain_filter_columns(
#     cols=data.columns.tolist(),
#     include_patterns=[ r'.*',],
#     # include_patterns=[ r'SphereA0' ,r'Bf0I1', r'psi_ps', r'coef'],
#     # exclude_patterns=[]
#     exclude_patterns=[r't\(M\)']
# )

visual_data = np.log10(data)

column_names = list(visual_data.columns)
column_names = return_sorted_domain_names(column_names)
visual_data = visual_data[column_names].iloc[remove_first_n_coeffs:,:]

vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()
print(vmin_log,",",vmax_log)


plt.figure(figsize=(15, 10))
# plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data, 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
    vmin=vmin_log, 
    vmax=vmax_log
)

# Add text annotations to each cell
for i in range(visual_data.shape[0]):
    for j in range(visual_data.shape[1]):
        # Choose text color based on cell value for better visibility
        value = visual_data.iloc[i, j]
        if np.isnan(value):
           continue
        # color = 'white' if value > (vmin_log + vmax_log)/2 else 'black'
        color = 'black'
        # Add text annotation
        plt.text(j, i, f'{value:.2f}',  # Scientific notation
                ha='center', 
                va='center', 
                color=color,rotation=90,
                fontsize=8)  # Adjust fontsize as needed

# Set x-ticks and labels
plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

plt.yticks(
    ticks=np.arange(0, len(visual_data), 1), 
    labels=np.arange(remove_first_n_coeffs, remove_first_n_coeffs+ len(visual_data), 1), 
)

# Create colorbar
colorbar = plt.colorbar(imshow_plot, label=f'{colorbar_label}')
plt.ylabel('Power in Coeffs')
plt.title(f'{PowDiag_path.parent}\n Time={time:.2f} , {irr_top_num=}')
plt.tight_layout() 
plt.grid(False)
plt.show()

##### all on a single plot

In [ ]:
time = 7500
power_cutoff = -24 # remove all coeffs smaller than this. These are usually the filtered coeffs.
remove_first_n_coeffs = (0,0,0)
remove_first_n_coeffs = (8,8,8)
time,series = series_closest_to_time(time,filtered_df)

psi_or_kappa = None
if 'psi_' in filtered_cols[0]:
    psi_or_kappa = 'psi'
else:
    psi_or_kappa = 'kappa'


fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(15, 15), constrained_layout=True)

for i, ax in zip(range(3),axes):

    data = convert_series_to_coeff_df(series,i)
    colorbar_label = f"{psi_or_kappa}(Extent {i}) : power coeff"

    visual_data = np.log10(data)

    visual_data[visual_data<power_cutoff] = 1e-20
    # visual_data = visual_data[visual_data>power_cutoff]

    column_names = list(visual_data.columns)
    column_names = return_sorted_domain_names(column_names)
    visual_data = visual_data[column_names].iloc[remove_first_n_coeffs[i]:,:]

    vmin_log,vmax_log = None,None
    if vmin_log is None:
        vmin_log = visual_data.min().min()
    if vmax_log is None:
        vmax_log = visual_data.max().max()
    print(vmin_log,",",vmax_log)

    imshow_plot = ax.imshow(
        visual_data, 
        aspect='auto', 
        cmap='RdYlGn_r', 
        origin='lower',interpolation='none',
        vmin=vmin_log, 
        vmax=vmax_log
    )

    if i == 2:
        # Set x-ticks and labels
        ax.set_xticks(
            ticks=np.arange(len(visual_data.columns)), 
            labels=[i.split(" ")[-1] for i in column_names], 
            rotation=90
        )

    ax.set_yticks(
        ticks=np.arange(0, len(visual_data), 1), 
        labels=np.arange(remove_first_n_coeffs[i], remove_first_n_coeffs[i]+ len(visual_data), 1), 
    )

    # ax.set_title(f"{column_name[i]}, {filter_suffix[-1]}")

    # Create colorbar
    # fig.colorbar(imshow_plot, label=f'{colorbar_label}',ax=ax)
    plt.colorbar(imshow_plot, label=f'{colorbar_label}',ax=ax)
    ax.set_ylabel('Power in Coeffs')
    ax.grid(False)
 
plt.suptitle(f'{PowDiag_path.parent}\n Time={time:.2f}, {power_cutoff=}')
plt.show()

### df dict

In [ ]:
PowDiag_path = Path("/groups/sxs/hchaudha/spec_runs/16_set1_L3/h5_files_Lev3/extracted-PowerDiagnostics")
dict_df = load_power_diagonistics(PowDiag_path)
dict_df.keys()

In [ ]:
# filter subdomains
# include_list = [ r'SphereA[2-9]']
include_list = [ ]
exclude_list = [  r'Cylinder',r'SphereC']
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=dict_df.keys(),
    include_patterns=include_list,
    exclude_patterns=exclude_list
)
filtered_dict = {key:dict_df[key] for key in filtered_cols}
top_set = set()
for k,i in filtered_dict.items():
  top_set.update(i.keys())
print(filtered_dict.keys())
print(top_set)

In [ ]:
# INCOMPLETE!!!!!
# filter subdomains
include_list = [ 'Bf0I1']
exclude_list = [ ]
# Example usage with a DataFrame
filtered_cols = chain_filter_columns(
    cols=dict_df.keys(),
    include_patterns=include_list,
    exclude_patterns=exclude_list
)
filtered_dict = {key:dict_df[key] for key in filtered_cols}
filtered_dict.keys()

## Exp damping 

In [ ]:
def exp_damping(p,a,N):
  return np.exp(-a*(np.arange(N)/(N-1))**(2*p))

In [ ]:
1-exp_damping(36,36,15), 1-exp_damping(30,36,15), 1-exp_damping(28,36,15)

In [ ]:
plt.plot(1-exp_damping(28,36,15),label='28,36', marker='o')
plt.plot(1-exp_damping(30,36,15),label='30,36', marker='o')
plt.plot(1-exp_damping(32,36,15),label='32,36', marker='o')
plt.plot(1-exp_damping(36,36,15),label='36,36', marker='o')
plt.yscale('log')
plt.legend()

In [ ]:
import cProfile
profiler = cProfile.Profile()
profiler.enable()
profiler.disable()

# Print sorted results
profiler.print_stats(sort='cumulative')

In [ ]:
re.escape("""      BjorhusCharSpeedCutoff = -1.e-10;
      InternalBdryMethod     = MultiPenalty;""")